# 02 — SUSY Signal Regions (CMS Derived ROOT: DY/WJets/TT vs SMS‑TChiWZ)

This notebook continues the previous EDA, focusing on *signal-like* selections.
We load CMS-derived ROOT features for DY, WJets, TT (background) and SMS‑TChiWZ_ZToLL (signal), apply baseline + physics cleaning, and define control/signal regions for fast validation plots.


## Quick validation checklist (pass/fail)

Before moving to training, confirm:

1) Inputs loaded:
- DY/WJets/TT/SUSY have non-zero rows
- ROOT tree name is detected (typically `Events`)

2) Cleaning is sensible:
- Event counts decrease only where expected (outliers/trivial events)

3) Regions are populated:
- Baseline has enough statistics
- Signal and control regions are both non-empty

4) Plot sanity:
- `signal_vs_control_met.jpg` is produced and shows a harder MET tail in the signal region by construction.


In [1]:
# Cell 1 — Install deps (no XRootD needed)
!pip -q install "uproot>=5" awkward vector rich tqdm pandas pyarrow fastparquet matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.7/656.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.3 MB/s eta 0:00:00


In [2]:
import uproot
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt


## 1) Feature schema

We use a fixed list of ~50 engineered observables spanning:
- Object multiplicities (`nMuon`, `nElectron`, `nJet`, `nJet_pt30`)
- MET (`MET_pt`, `MET_phi`, `MET_sumEt`)
- Leading muon/electron/jet kinematics (pt/eta/phi for indices 0–3 where applicable)
- Event summaries (`HT`, `ST`)
- Dilepton/dijet masses (`M_ll`, `M_jj_01`, `M_jj_12`)
- Angular variables (`delta_phi_*`, `delta_R_*`)
- b-tag scores (`Jet_btagDeepB_0/1`)
- Ratios (`HT_ratio`, `MET_pt_HT_ratio`)


In [3]:
FEATURES = [
"nMuon","nElectron","nJet","MET_pt","MET_phi","MET_sumEt",
"Muon_pt_0","Muon_eta_0","Muon_phi_0",
"Muon_pt_1","Muon_eta_1","Muon_phi_1",
"Electron_pt_0","Electron_eta_0","Electron_phi_0",
"Electron_pt_1","Electron_eta_1","Electron_phi_1",
"Jet_pt_0","Jet_eta_0","Jet_phi_0",
"Jet_pt_1","Jet_eta_1","Jet_phi_1",
"Jet_pt_2","Jet_eta_2","Jet_phi_2",
"Jet_pt_3","Jet_eta_3","Jet_phi_3",
"HT","ST","M_ll","M_jj_01","M_jj_12",
"delta_phi_MET_j0","delta_phi_MET_j1","min_delta_phi_MET_jets",
"delta_R_j0_j1","delta_phi_ll","delta_R_ll",
"Jet_btagDeepB_0","Jet_btagDeepB_1",
"MT_lep_MET","HT_ratio","MET_pt_HT_ratio",
"nJet_pt30","Jet_mass_0","LeadLepton_pt","sum_pt_leptons"
]


## 2) Loading approach (fast + robust)

We read ROOT files with `uproot` and:
1. Recursively find the first available TTree in each file (usually `Events`)
2. Iterate in chunks (`step_size`) to control memory
3. Convert each batch to pandas and append `label` + `source`
4. Concatenate into a single dataframe for cleaning + selections

We cap `max_files` during development; increase it once plots and selections look stable.


In [4]:
import uproot, pandas as pd, glob, gc
import pyarrow as pa
import pyarrow.parquet as pq

def stream_folder_to_parquet(
        folder,
        out_path,
        label,
        source,
        step_size=50000):

    files = sorted(glob.glob(folder+"/*.root"))

    print(f"\nStreaming {source} from {len(files)} files")

    writer = None
    total = 0

    for i, f in enumerate(files):

        print(f"File {i+1}/{len(files)}")

        with uproot.open(f) as file:

            tree = None

            # detect tree automatically
            for key, obj in file.items(recursive=True):
                if isinstance(obj, uproot.behaviors.TTree.TTree):
                    tree = obj
                    break

            if tree is None:
                print("   No tree → skip")
                continue

            print("   Tree:", tree.name)

            for batch in tree.iterate(FEATURES, library="pd", step_size=step_size):

                batch["label"] = label
                batch["source"] = source

                table = pa.Table.from_pandas(batch)

                if writer is None:
                    writer = pq.ParquetWriter(out_path, table.schema)

                writer.write_table(table)

                total += len(batch)

                # free memory
                del batch
                gc.collect()

    if writer:
        writer.close()

    print(f"Saved {total} events → {out_path}")

In [5]:
import uproot, pandas as pd, glob, gc, awkward as ak
import pyarrow as pa
import pyarrow.parquet as pq

def stream_real_rntuple(folder, out_path, step_size=50000):

    files = sorted(glob.glob(folder+"/*.root"))
    print(f"\nStreaming REAL (RNTuple) from {len(files)} files")

    writer = None
    total = 0

    for i, f in enumerate(files):

        print(f"File {i+1}/{len(files)}")

        with uproot.open(f) as file:

            if "Features" not in file:
                print("   Features not found → skip")
                continue

            ntuple = file["Features"]
            print("   Found RNTuple")

            # iterate batches as awkward arrays
            for batch in ntuple.iterate(step_size=step_size):

                # convert awkward → pandas safely
                batch = ak.to_dataframe(batch).reset_index(drop=True)

                # keep only feature columns
                batch = batch[FEATURES]

                batch["label"] = -1
                batch["source"] = "REAL"

                table = pa.Table.from_pandas(batch)

                if writer is None:
                    writer = pq.ParquetWriter(out_path, table.schema)

                writer.write_table(table)

                total += len(batch)

                del batch
                gc.collect()

    if writer:
        writer.close()

    print(f"Saved {total} REAL events → {out_path}")

In [6]:
import glob

base_real = "/kaggle/input/datasets/hiteshrs/cms2016g29-5785/processed_events"

files = sorted(glob.glob(base_real+"/*.root"))

print("Number of files:", len(files))
print("Example file:", files[0])

Number of files: 85
Example file: /kaggle/input/datasets/hiteshrs/cms2016g29-5785/processed_events/DerivedFeatures(1).root


In [7]:
import uproot

with uproot.open(files[0]) as f:

    print("=== KEYS (recursive) ===")
    print(f.keys(recursive=True))

    print("\n=== CLASSNAMES ===")
    print(f.classnames(recursive=True))

=== KEYS (recursive) ===
['Features;1']

=== CLASSNAMES ===
{'Features;1': 'ROOT::RNTuple'}


In [8]:
with uproot.open(files[0]) as f:

    print("\n=== OBJECT TYPES ===")

    for key, obj in f.items(recursive=True):
        print(key, "->", type(obj))


=== OBJECT TYPES ===
Features;1 -> <class 'uproot.models.RNTuple.Model_ROOT_3a3a_RNTuple'>


In [9]:
with uproot.open(files[0]) as f:
    tree = f["Features"]
    print(tree)
    print(tree.keys())

<ROOT::RNTuple 'Features' (52 fields) at 0x780a70e267b0>
['Electron_eta_0', 'Electron_eta_1', 'Electron_phi_0', 'Electron_phi_1', 'Electron_pt_0', 'Electron_pt_1', 'HT', 'HT_ratio', 'Jet_btagDeepB_0', 'Jet_btagDeepB_1', 'Jet_eta_0', 'Jet_eta_1', 'Jet_eta_2', 'Jet_eta_3', 'Jet_mass_0', 'Jet_phi_0', 'Jet_phi_1', 'Jet_phi_2', 'Jet_phi_3', 'Jet_pt_0', 'Jet_pt_1', 'Jet_pt_2', 'Jet_pt_3', 'LeadLepton_pt', 'MET_phi', 'MET_pt', 'MET_pt_HT_ratio', 'MET_sumEt', 'MT_lep_MET', 'M_jj_01', 'M_jj_12', 'M_ll', 'Muon_eta_0', 'Muon_eta_1', 'Muon_phi_0', 'Muon_phi_1', 'Muon_pt_0', 'Muon_pt_1', 'ST', 'delta_R_j0_j1', 'delta_R_ll', 'delta_phi_MET_j0', 'delta_phi_MET_j1', 'delta_phi_ll', 'luminosityBlock', 'min_delta_phi_MET_jets', 'nElectron', 'nJet', 'nJet_pt30', 'nMuon', 'run', 'sum_pt_leptons']


## 3) Inputs

MC/SUSY (derived features, ROOT):
- `/kaggle/input/datasets/katakuricharlotte/cms-derivedroot/derivedroot/`
  - DYJetsToLL_0J_TuneCP5
  - WJetsToLNu_TuneCP5
  - TTJets_TuneCP5
  - SMS-TChiWZ_ZToLL

Optional real-data cross-check:
- `/kaggle/input/datasets/hiteshrs/cms2016g29-5785/processed_events`

Notes:
- If REAL loads as “0 files / No data loaded”, we treat it as unavailable and proceed with MC-only training tables.


In [ ]:
base_mc = "/kaggle/input/datasets/katakuricharlotte/cms-derivedroot/derivedroot"
base_real = "/kaggle/input/datasets/hiteshrs/cms2016g29-5785/processed_events"

# MC
stream_folder_to_parquet(f"{base_mc}/DYJetsToLL_0J_TuneCP5", "/kaggle/working/dy.parquet", 0, "DY")
stream_folder_to_parquet(f"{base_mc}/WJetsToLNu_TuneCP5", "/kaggle/working/wjets.parquet", 0, "WJets")
stream_folder_to_parquet(f"{base_mc}/TTJets_TuneCP5", "/kaggle/working/tt.parquet", 0, "TT")
stream_folder_to_parquet(f"{base_mc}/SMS-TChiWZ_ZToLL", "/kaggle/working/susy.parquet", 1, "SUSY")

# REAL (RNTuple)
stream_real_rntuple(base_real, "/kaggle/working/real.parquet")


Streaming DY from 4 files
File 1/4
   Tree: Events
File 2/4
   Tree: Events
File 3/4
   Tree: Events
File 4/4
   Tree: Events
Saved 6063865 events → /kaggle/working/dy.parquet

Streaming WJets from 4 files
File 1/4
   Tree: Events
File 2/4
   Tree: Events
File 3/4
   Tree: Events
File 4/4
   Tree: Events
Saved 5407002 events → /kaggle/working/wjets.parquet

Streaming TT from 4 files
File 1/4
   Tree: Events
File 2/4
   Tree: Events
File 3/4
   Tree: Events
File 4/4
   Tree: Events
Saved 3145070 events → /kaggle/working/tt.parquet

Streaming SUSY from 4 files
File 1/4
   Tree: Events
File 2/4
   Tree: Events
File 3/4
   Tree: Events
File 4/4
   Tree: Events
Saved 7827023 events → /kaggle/working/susy.parquet

Streaming REAL (RNTuple) from 85 files
File 1/85
   Found RNTuple


In [ ]:
stream_real_rntuple(
    "/kaggle/input/datasets/hiteshrs/cms2016g29-5785/processed_events",
    "/kaggle/working/real.parquet"
)

In [ ]:
import pandas as pd

df = pd.concat([
pd.read_parquet("/kaggle/working/dy.parquet"),
pd.read_parquet("/kaggle/working/wjets.parquet"),
pd.read_parquet("/kaggle/working/tt.parquet"),
pd.read_parquet("/kaggle/working/susy.parquet"),
pd.read_parquet("/kaggle/working/real.parquet")
], ignore_index=True)

print(df["source"].value_counts())

## 4) Cleaning and selections

Cleaning steps:
- Replace ±inf → NaN, drop NaNs
- Apply detector-realism bounds (e.g., MET and leading object pt upper caps)
- Apply eta acceptance (|eta| < 5 for all *_eta_* columns)
- Remove trivial events (require at least one jet or lepton)

Region definitions:
- Baseline: `MET_pt > 50`, `nJet >= 2`, `HT > 200`, `LeadLepton_pt > 20`
- Signal region: `MET_pt > 200`, `HT > 400`, `nJet >= 3`
- Control region: `MET_pt < 100`, `HT < 300`

Outputs:
- Diagnostic counts after each stage
- A first validation plot comparing MET in control vs signal region (log-y), saved to `/kaggle/working/`.


In [ ]:
df = pd.concat([dy, wj, tt, susy, real], ignore_index=True)

df_clean = df.copy()

print("Initial events:", len(df_clean))


In [ ]:
import numpy as np

df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
df_clean.dropna(inplace=True)

print("After NaN removal:", len(df_clean))


In [ ]:
# transverse momenta upper bounds (detector realism)
df_clean = df_clean[
    (df_clean["MET_pt"] < 2000) &
    (df_clean["Jet_pt_0"] < 3000) &
    (df_clean["Muon_pt_0"] < 2000) &
    (df_clean["Electron_pt_0"] < 2000)
]

# eta detector acceptance
eta_cols = [c for c in df_clean.columns if "_eta_" in c]

for c in eta_cols:
    df_clean = df_clean[df_clean[c].abs() < 5]

print("After physics cleaning:", len(df_clean))


In [ ]:
df_clean = df_clean[
    (df_clean["nJet"] > 0) |
    (df_clean["nMuon"] > 0) |
    (df_clean["nElectron"] > 0)
]

print("After trivial-event removal:", len(df_clean))


In [ ]:
baseline = df_clean[
    (df_clean["MET_pt"] > 50) &
    (df_clean["nJet"] >= 2) &
    (df_clean["HT"] > 200) &
    (df_clean["LeadLepton_pt"] > 20)
]

print("Baseline events:", len(baseline))


In [ ]:
signal_region = baseline[
    (baseline["MET_pt"] > 200) &
    (baseline["HT"] > 400) &
    (baseline["nJet"] >= 3)
]

print("Signal region events:", len(signal_region))


In [ ]:
control_region = baseline[
    (baseline["MET_pt"] < 100) &
    (baseline["HT"] < 300)
]

print("Control region events:", len(control_region))


In [ ]:
# Cell — Plot styling helper (define once)

import matplotlib as mpl

def paper_axes(ax,
              title=None,
              grid=True,
              grid_alpha=0.25,
              spine_width=1.2,
              tick_width=1.2,
              label_size=11,
              tick_size=10):
    """Lightweight 'paper' style formatting for a Matplotlib axis."""
    if title is not None:
        ax.set_title(title, fontsize=label_size + 1)

    # Spines
    for s in ax.spines.values():
        s.set_linewidth(spine_width)

    # Ticks
    ax.tick_params(axis="both", which="major", labelsize=tick_size,
                   width=tick_width, length=5, direction="in")
    ax.tick_params(axis="both", which="minor",
                   width=tick_width, length=3, direction="in")

    # Labels (keep whatever text you already set; just size them)
    ax.xaxis.label.set_size(label_size)
    ax.yaxis.label.set_size(label_size)

    # Grid
    if grid:
        ax.grid(True, which="major", alpha=grid_alpha, linewidth=0.8)
        ax.grid(True, which="minor", alpha=grid_alpha * 0.6, linewidth=0.5)

    return ax


In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=120)

ax.hist(control_region["MET_pt"], bins=100, histtype="step",
        linewidth=1.8, density=True, label="Control")

ax.hist(signal_region["MET_pt"], bins=100, histtype="step",
        linewidth=1.8, density=True, label="Signal region")

ax.set_yscale("log")
ax.set_xlabel("MET_pt [GeV]")
ax.set_ylabel("Normalized events")
ax.legend()

paper_axes(ax)
fig.tight_layout()

fig.savefig("/kaggle/working/signal_vs_control_met.jpg", dpi=300)
plt.show()


In [ ]:
train_df = baseline[baseline["label"] != -1]

X = train_df.drop(columns=["label","source"])
y = train_df["label"]

print("Training events:", len(X))


## 6) Next step (modeling-ready table)

For training:
- Use `baseline` events
- Exclude REAL rows (`label != -1`)
- Define `X = features` and `y = label`

This produces a clean binary classification table (background vs SUSY) ready for a baseline model.
